In [ ]:
!nvidia-smi

Wed Jan  4 01:05:32 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   54C    P0    28W /  70W |  11524MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime : {:.1f} GB.'.format(ram_gb))


Your runtime : 27.3 GB.


In [ ]:
!pip install -U pip transformers
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# hf_checkpoint = 'facebook/nllb-200-1.3B'
# hf_checkpoint = 'facebook/nllb-200-3.3B'
# hf_checkpoint = 'facebook/nllb-200-distilled-600M'
# hf_checkpoint = 'facebook/nllb-200-distilled-1.3B'
# hf_checkpoint = 'thefrigidliquidation/nllb-jaen-1.3B-lightnovels'
# hf_checkpoint = 'thefrigidliquidation/nllb-200-distilled-1.3B-bookworm'

In [ ]:
ht_ckp = [
# 'facebook/nllb-200-3.3B',
'facebook/nllb-200-1.3B',
'facebook/nllb-200-distilled-600M',
'facebook/nllb-200-distilled-1.3B',
'thefrigidliquidation/nllb-jaen-1.3B-lightnovels',
'thefrigidliquidation/nllb-200-distilled-1.3B-bookworm',   
]

# 文章の出典（一部引用）
# https://news.yahoo.co.jp/pickup/6449584 2023.1.4配信より
# https://bwgift.hatenablog.com/
# https://news.yahoo.co.jp/articles/87745cf9da199bc5c2f1dc3871144214f0f8c8f4 2023.1.3. 6時 配信より
# https://ja.wikipedia.org/wiki/%E8%87%AA%E7%84%B6%E8%A8%80%E8%AA%9E%E5%87%A6%E7%90%86

trg_texts = [
    "北海道や東北は日本海側を中心に雪が降り、ふぶく所もあるでしょう。これまでの大雪で積雪が急増している所があります。屋根から落ちてくる雪やなだれに注意してください。",
    "最初は、つかみどころのない映画だなと見始めたが、素敵な音楽が散りばめられた音楽と青春映画だった。「映画」としてみると評価はしにくいけれども、音楽と青春のもどかしさを愛でる作品として素敵な印象を残した。",
    "今夜日付が変わった後、4日(水)の未明から明け方にかけて、三大流星群のひとつ「しぶんぎ座流星群」が見頃を迎えます。",
    "自然言語処理（しぜんげんごしょり、英語: natural language processing、略称：NLP）は、人間が日常的に使っている自然言語をコンピュータに処理させる一連の技術であり、人工知能と言語学の一分野である。",
 ]


In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
import torch

device = 0 if torch.cuda.is_available() else -1


In [ ]:
for checkpoint in ht_ckp:
  tokenizer = AutoTokenizer.from_pretrained(checkpoint)
  model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)
  input_lang='jpn_Jpan'
  target_lang = 'eng_Latn'
  translation_pipeline = pipeline('translation', 
                                  model=model, 
                                  tokenizer=tokenizer, 
                                  src_lang=input_lang, 
                                  tgt_lang=target_lang, 
                                  max_length = 400,
                                  device=device)
  
  for text in trg_texts:
    %time output = translation_pipeline(text)
    print(checkpoint, output[0]['translation_text'])

CPU times: user 1.27 s, sys: 12.7 ms, total: 1.29 s
Wall time: 1.28 s
facebook/nllb-200-1.3B There are places where snow falls and blows, especially in the northern part of the Hokkaido and Tōhoku regions, and on the Japanese side. There are places where snow has been increasing rapidly due to heavy snowfall. Be careful of snow falling from the roof.
CPU times: user 1.29 s, sys: 2.63 ms, total: 1.29 s
Wall time: 1.29 s
facebook/nllb-200-1.3B At first, I thought it was a film that was hard to find, but it was a film of music and youth, with beautiful music scattered around it. It was difficult to evaluate it as a "film", but it left a good impression as a work that loves music and youthfulness.
CPU times: user 807 ms, sys: 3.84 ms, total: 811 ms
Wall time: 808 ms
facebook/nllb-200-1.3B After the date change tonight, from the 4th of October, at dawn, we will see one of the three major meteor clusters, the "Cosmic Cluster".
CPU times: user 770 ms, sys: 0 ns, total: 770 ms
Wall time: 766 m

# Google翻訳  
- "In Hokkaido and Tohoku, it will snow mainly on the Sea of ​​Japan side, and there will be heavy snowfall. There are places where the snowfall is increasing rapidly due to the heavy snowfall. Be careful of snow and avalanches falling from the roof. Please give me.",

- "At first, I thought it was an elusive movie, but it turned out to be a music and youth film with wonderful music. It left a wonderful impression as a beloved work. ",

- After the date changes tonight, the Quadrantid meteor shower, one of the three major meteor showers, will reach its peak from dawn to dawn on Wednesday, April 4.

- Natural language processing (English: natural language processing, abbreviation: NLP) is a series of technologies that allow computers to process the natural language that humans use on a daily basis, and is a part of artificial intelligence and linguistics. field.

In [1]:
# 参考文献
# https://huggingface.co/thefrigidliquidation
# https://huggingface.co/spaces/Narrativaai/NLLB-Translator/blob/main/app.py